# Conformal Learning Demo (Ex-Fuzzy)

This notebook demonstrates split conformal prediction with Ex-Fuzzy:

- Train a fuzzy rule-based classifier
- Calibrate on a held-out calibration set
- Produce set-valued predictions with target coverage `1 - alpha`
- Inspect rule-aware conformal explanations

In [ ]:
import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split

import ex_fuzzy.fuzzy_sets as fs
import ex_fuzzy.utils as utils
from ex_fuzzy.conformal import ConformalFuzzyClassifier, evaluate_conformal_coverage

In [ ]:
# Parameters
N_GEN = 20
POP_SIZE = 30
N_RULES = 15
N_ANTS = 3
N_LV = 3
FUZZY_TYPE = fs.FUZZY_SETS.t1
TOLERANCE = 0.01
ALPHA = 0.10

# Load Iris data
iris = datasets.load_iris()
X = pd.DataFrame(iris.data, columns=iris.feature_names)
y = iris.target

# Split into train/calibration/test
X_train_full, X_test, y_train_full, y_test = train_test_split(
    X, y, test_size=0.25, random_state=0, stratify=y
)
X_train, X_cal, y_train, y_cal = train_test_split(
    X_train_full, y_train_full, test_size=0.30, random_state=0, stratify=y_train_full
)

# Build fuzzy partitions from train split
precomputed_partitions = utils.construct_partitions(
    X_train, FUZZY_TYPE, n_partitions=N_LV
)

# Fit + calibrate conformal classifier
conf_clf = ConformalFuzzyClassifier(
    nRules=N_RULES,
    nAnts=N_ANTS,
    fuzzy_type=FUZZY_TYPE,
    tolerance=TOLERANCE,
    linguistic_variables=precomputed_partitions,
    verbose=False,
)
conf_clf.fit(
    X_train,
    y_train,
    X_cal,
    y_cal,
    n_gen=N_GEN,
    pop_size=POP_SIZE,
    random_state=0,
)

# Evaluate conformal coverage
metrics = evaluate_conformal_coverage(conf_clf, X_test, y_test, alpha=ALPHA)
pred_sets = conf_clf.predict_set(X_test, alpha=ALPHA)
explained = conf_clf.predict_set_with_rules(X_test[:3], alpha=ALPHA)

print(f"Expected coverage: {metrics['expected_coverage']:.3f}")
print(f"Empirical coverage: {metrics['coverage']:.3f}")
print(f"Average set size: {metrics['avg_set_size']:.3f}")
print(f"Singleton sets: {metrics['singleton_sets']:.3f}")
print(f"Empty sets: {metrics['empty_sets']:.3f}")
print("Coverage by class:", {int(k): float(v) for k, v in metrics['coverage_by_class'].items()})

print("\nFirst 5 prediction sets:")
for i, pred_set in enumerate(pred_sets[:5]):
    print(f"Sample {i}: {pred_set}")

print("\nRule-aware conformal explanations (first 3 samples):")
for i, item in enumerate(explained):
    print(f"Sample {i} prediction set: {item['prediction_set']}")
    class_p_values = {int(k): float(v) for k, v in item['class_p_values'].items()}
    print(f"Sample {i} class p-values: {class_p_values}")
    for contrib in item['rule_contributions'][:3]:
        print(
            f"  Rule {contrib['rule_index']} -> class {contrib['class']}, "
            f"strength={contrib['firing_strength']:.3f}, confidence={contrib['rule_confidence']:.3f}"
        )